In [2]:
import re
from typing import List, Dict

from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
HIERARCHY = ["livre", "titre", "chapitre", "section", "sous_section"]

def update_context(context, level, value):
    """
    Update parsing context and reset lower levels if needed.
    """
    context[level] = value
    idx = HIERARCHY.index(level)
    for lower in HIERARCHY[idx+1:]:
        context[lower] = ''
    return context

def parse_ceseda(text: str) -> List[Dict]:
    """
    Découpe le CESEDA en articles avec contexte hiérarchique et références.
    Hérite des infos (titre, chapitre, section, sous-section) de l’article précédent
    si elles ne sont pas trouvées explicitement.
    """
    # Regex hiérarchie
    livre_pattern = re.compile(r"(Livre\s+[IVXLC]+\s*:.*)")
    titre_pattern = re.compile(r"(Titre\s+[IVXLC]+\s*:.*)")
    chapitre_pattern = re.compile(r"(Chapitre\s+.*)")
    section_pattern = re.compile(r"(Section\s+\d+\s*:.*)")
    sous_section_pattern = re.compile(r"(Sous-section\s+\d+\s*:.*)")
    
    # Regex article
    article_pattern = re.compile(r"(Article\s+[LR]\.?\s*\d+[-\d]*\s*:?)")
    
    # Regex références internes
    reference_pattern = re.compile(r"article\s+([LR]\.?\s*\d+[-\d]*)")
    
    articles: Dict = {}
    
    # Découpage par articles
    parts = re.split(article_pattern, text)
    
    for i in range(1, len(parts), 2):
        header = parts[i].strip()
        body = parts[i+1].strip() if i+1 < len(parts) else ""
        before = parts[i-1]  # texte avant l’article
        
        # Récupération du contexte de l’article précédent si existant
        prev = articles[list(articles.keys())[-1]] if articles else {}
        current_livre = prev.get("livre", "")
        current_titre = prev.get("titre", "")
        current_chapitre = prev.get("chapitre", "")
        current_section = prev.get("section", "")
        current_sous_section = prev.get("sous_section", "")

        current_dict = {
            "livre": current_livre,
            "titre": current_titre,
            "chapitre": current_chapitre,
            "section": current_section,
            "sous_section": current_sous_section
        }
        
        # Mise à jour seulement si trouvé dans "before" if we change section or chapitre or titre or livre we don't want to keep the previous one
        livre_match = livre_pattern.search(before)
        if livre_match:
            current_dict = update_context(current_dict, "livre", livre_match.group(1).strip())  

        titre_match = titre_pattern.search(before)
        if titre_match:
            current_dict = update_context(current_dict, "titre", titre_match.group(1).strip())
        
        chapitre_match = chapitre_pattern.search(before)
        if chapitre_match:
            current_dict = update_context(current_dict, "chapitre", chapitre_match.group(1).strip())
        
        section_match = section_pattern.search(before)
        if section_match:
            current_dict = update_context(current_dict, "section", section_match.group(1).strip())
        
        sous_section_match = sous_section_pattern.search(before)
        if sous_section_match:
            current_dict = update_context(current_dict, "sous_section", sous_section_match.group(1).strip())
        
        # Identifiant de l’article
        article_num = re.search(r"([LR]\.?\s*\d+[-\d]*)", header)
        art_code = article_num.group(1).replace(" ", "") if article_num else header
        
        # Références citées
        refs = [m.group(1).replace(" ", "") for m in reference_pattern.finditer(body)]
        
        articles[art_code] = {
            **current_dict,
            "content": body,
            "referenced": refs
        }
    
    return articles

def summerize(text: str, summarizer = summarizer, max_length: int = 500) -> str:
    """
    Summarize the given text using a pre-trained transformer model.
    """
    # Summarize just the article
    summary_article = summarizer(text, max_length=max_length, min_length=30, do_sample=False)[0]['summary_text']

    return summary_article

def parsed_with_summaries(articles: Dict, max_length: int = 500) -> Dict:
    """
    Add summaries to each article in the parsed CESEDA
    3 levels of summarization
    input: article text + referenced articles + context (title, chapter, section, sub-section)
    output: 
        - summarized text for just the article (no context, no references)
        - summarized text for the article with context 
        - summarized text for the article with context and references
    
    """
    for art_code, art_data in articles.items():
        art_content = art_data["content"]
        art_summary = summerize(art_content, max_length=max_length)
        articles[art_code]["summary_level0"] = art_summary
        # Add context
        context = " ".join(filter(None, [art_data["livre"], art_data["titre"], art_data["chapitre"], art_data["section"], art_data["sous_section"]]))
        art_with_context = context + " " + art_content
        art_summary_context = summerize(art_with_context, max_length=max_length)
        articles[art_code]["summary_level1"] = art_summary_context
        # Add referenced articles
        referenced_texts = " ".join([articles[ref]["content"] for ref in art_data["referenced"] if ref in articles])
        art_with_references = art_with_context + " " + referenced_texts
        art_summary_references = summerize(art_with_references, max_length=max_length)
        articles[art_code]["summary_level2"] = art_summary_references
    return articles


Device set to use cpu


In [7]:
# Charger ton CESEDA brut (extrait ou texte complet)
import pypdf
from pypdf import PdfReader

# creating a pdf reader object
reader = PdfReader('sources\LEGITEXT000006070158.pdf')

# get page text without footer and header
def get_page_text(page, header_lines=0, footer_lines=1):
    text = page.extract_text()
    lines = text.split("\n")
    content_lines = lines[:-footer_lines] if footer_lines > 0 else lines
    content_lines = content_lines[header_lines:] if header_lines > 0 else content_lines
    return "\n".join(content_lines)

# # Parsing all the pages of pdf to extract text
# text = ""
# for page in reader.pages:
#     text += get_page_text(page) + "\n"
# articles = parse_ceseda(text)
# # Exemple d’utilisation
# for art_id, art in list(articles.items())[:10]:
#     print(f"{art_id}: {art['titre']} - {art['chapitre']}")
#     print(f"chapitre: {art['chapitre']}")
#     print(f"section: {art['section']}")
#     print(f"sous-section: {art['sous_section']}")
#     print(f"livre: {art['livre']}")
#     print(f"Contenu: {art['content'][:100]}...")
#     print(f"Références: {art['referenced']}")
#     print("-----")

<>:6: SyntaxWarning: invalid escape sequence '\L'
<>:6: SyntaxWarning: invalid escape sequence '\L'
C:\Users\sofia\AppData\Local\Temp\ipykernel_47628\1394744437.py:6: SyntaxWarning: invalid escape sequence '\L'
  reader = PdfReader('sources\LEGITEXT000006070158.pdf')


In [ ]:
# test summarization
articles = parsed_with_summaries(articles, max_length=50)
print(articles['L.123-1'])


In [24]:
# save articles to a json file
import json

with open('sources/parsed/ceseda_articles.json', 'w', encoding='utf-8') as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)

In [11]:
# reading JOURNAL OFFICIEL Republique Française

import pypdf
from pypdf import PdfReader


jorf = PdfReader('sources\JORF_19581005_234.pdf')
# example of reading the text of the first page
text_jorf = jorf.pages[2]
print(text_jorf.extract_text()[:500])

# parsing the jorf using the same function as ceseda
text_jorf_full = ""
for page in jorf.pages[1:]:
    text_jorf_full += get_page_text(page, header_lines=1, footer_lines=0) + "\n"
articles_jorf = parse_ceseda(text_jorf_full)
# Exemple d’utilisation
for art_id, art in list(articles_jorf.items())[:10]:
    print(f"{art_id}: {art['titre']} - {art['chapitre']}")
    print(f"chapitre: {art['chapitre']}")
    print(f"section: {art['section']}")
    print(f"sous-section: {art['sous_section']}")
    print(f"livre: {art['livre']}")
    print(f"Contenu: {art['content'][:100]}...")
    print(f"Références: {art['referenced']}")
    print("-----")

<>:7: SyntaxWarning: invalid escape sequence '\J'
<>:7: SyntaxWarning: invalid escape sequence '\J'
C:\Users\sofia\AppData\Local\Temp\ipykernel_47628\1438572154.py:7: SyntaxWarning: invalid escape sequence '\J'
  jorf = PdfReader('sources\JORF_19581005_234.pdf')


5 Octobre 1058 JOURNAL OFFICIEL DE LA REPUBLIQUE FRANÇAISE 9151
CONSTITUTION
Le Gouvernement de la République, conformément à la loi 
constitutionnelle du 3 juin 1958, a proposé,
Le Peuple français a adopté,
Le Président de la République promulgue la loi constitution­
nelle dont la teneur suit :
PREAMBULE
Le peuple français proclame solennellement son attachement aux Droits de l’homme et aux 
principes de la souveraineté nationale tels qu’ils ont été définis par la Déclaration de 1789, 
Confirmé


In [ ]:
articles_jorf 

{}

In [34]:
article_pattern = re.compile(r"(Article\s+[A-Za-z\-]*+[-\d]*\s*:?)")
reference_pattern = re.compile(r"article\s+(?:([LR])\.?\s*)?(\d+(?:-\d+)*)")
titre_pattern = re.compile(r"(Titre\s+(?:[IVXLC]+|PREMIER|DEUXIÈME|TROISIÈME|QUATRIÈME|CINQUIÈME|SIXIÈME|SEPTIÈME|HUITIÈME|NEUVIÈME|DIXIÈME)\s*:?.*)", re.IGNORECASE)
parts = re.split(titre_pattern, text_jorf_full)
for p in parts[:10]:
    print("----")
    print(p)

----
\
'
♦ . .
'■Su ubi.
■ ; ■ ' ; - : . A
.. W. ■ • : :>
'■ « • ■ -, ■■ • ■ : ; •. ’
r‘
________________

CONSTITUTION
Le Gouvernement de la République, conformément à la loi 
constitutionnelle du 3 juin 1958, a proposé,
Le Peuple français a adopté,
Le Président de la République promulgue la loi constitution­
nelle dont la teneur suit :
PREAMBULE
Le peuple français proclame solennellement son attachement aux Droits de l’homme et aux 
principes de la souveraineté nationale tels qu’ils ont été définis par la Déclaration de 1789, 
Confirmée et complétée par le préambule de la Constitution de 1946.
En vertu de ces principes et de celui de la libre détermination des peuples, la République 
offre aux territoires d’Outre-Mer qui manifestent la volonté d’y adhérer des institutions 
nouvelles fondées sur l’idéal commun de liberté, d’égalité et de fraternité et conçues en vue de 
leur évolution démocratique.
Article premier.
La République et les peuples des territoires d’Outre-Mer qui, par un a

In [35]:
# test pattern on legal text
text_ceseda = ""
for page in reader.pages:
    text_ceseda += get_page_text(page, header_lines=0, footer_lines=1) + "\n"
parts_ceseda = re.split(titre_pattern, text_ceseda)
for p in parts_ceseda[:20]:
    print("----")
    print(p)

----
Code de l'entrée et du séjour des étrangers et du droit d'asile
Partie législative
Livre I : DISPOSITIONS GÉNÉRALES

----
Titre I : CHAMP D'APPLICATION
----

Article L110-1
  
Le présent code régit, sous réserve du droit de l'Union européenne et des conventions internationales,
l'entrée, le séjour et l'éloignement des étrangers en France ainsi que l'exercice du droit d'asile.
Article L110-2
  
Le présent code est applicable sur l'ensemble du territoire de la République.
Article L110-3
  
Sont considérées comme étrangers au sens du présent code les personnes qui n'ont pas la nationalité
française, soit qu'elles aient une nationalité étrangère, soit qu'elles n'aient pas de nationalité.
Article L110-4
  
Sans préjudice du droit de l'Union européenne, le livre II du présent code régit l'entrée, le séjour et
l'éloignement des citoyens de l'Union européenne, des étrangers qui leur sont assimilés ainsi que des
étrangers membres de leur famille ou entretenant avec eux des liens privés et 

In [36]:
a = ["jkd", "jjkd"]
a[0:None]

['jkd', 'jjkd']

In [1]:
from parsing import parse_pdf
import json

metadata = json.load(open("sources/metadata.json"))

file_dict = metadata["jorf_19581005"]
articles = parse_pdf(file_dict)

for art_id, art in list(articles.items()):
    print(f"{art_id}")
    print(f"titre: {art['titre']}")
    print(f"chapitre: {art['chapitre']}")
    print(f"section: {art['section']}")
    print(f"sous-section: {art['sous_section']}")
    print(f"livre: {art['livre']}")
    print(f"Contenu: {art['content'][:100]}...")
    print(f"Références: {art['referenced']}")
    print("-----")

Device set to use cpu


Article prem1er
titre: 
chapitre: 
section: 
sous-section: 
livre: 
Contenu: .
La République et les peuples des territoires d’Outre-Mer qui, par un acte de libre détermi­
nation...
Références: []
-----
Article 2
titre: TITRE PREMIER
DE LA SOUVERAINETE
chapitre: 
section: 
sous-section: 
livre: 
Contenu: .
\
La France est une République indivisible, laïque, démocratique et sociale. Elle assure 
l’égalit...
Références: []
-----
Article 4
titre: TITRE PREMIER
DE LA SOUVERAINETE
chapitre: 
section: 
sous-section: 
livre: 
Contenu: .
Les partis et groupements politiques concourent à l’expression du suffrage. Ils se forment 
et exe...
Références: []
-----
Article 5
titre: TITRE II
LE PRESIDENT DE LA REPUBLIQUE
chapitre: 
section: 
sous-section: 
livre: 
Contenu: .
Le Président de la République veille au respect de la Constitution. Il assure, par son 
arbitrage,...
Références: []
-----
Article 6
titre: TITRE II
LE PRESIDENT DE LA REPUBLIQUE
chapitre: 
section: 
sous-section: 
livre: 
Contenu:

In [13]:
text = "articles 11 et 12"
import re

reference_pattern = re.compile(r"articles?\s+([-\dA-Za-z]+)", re.IGNORECASE)

re.findall(reference_pattern, text)

['11']

In [10]:
for m in reference_pattern.finditer(text):
    print(m.group(2))

11


In [17]:
import re

text = "articles 11 et 12, 13A  ofeoiejlfz L-1909 nfzjflqeij 2002"

# This regex matches *all numbers/letters/hyphens* after 'article(s)'
pattern = re.compile(r'\b\d+[A-Za-z-]*\b')

# First, find the part after "article(s)"
articles_part = re.search(r'articles?\s+(.*)', text, re.IGNORECASE)
if articles_part:
    # Extract all numbers from that part
    refs = pattern.findall(articles_part.group(1))
    print(refs)


['11', '12', '13A', '1909', '2002']


In [2]:
text = 'go'

translation_table = str.maketrans({
    " ": "",  # remove spaces
    "g": "9",
    "i": "1",
    "o": "0"
})
text.translate(translation_table)

'90'

In [ ]:
import re
# test titre pattern why can't detect the last titre
titre_pattern = re.compile(r"(Titre\s+(?:[IVXLC]+|PREMIER|DEUXIÈME|TROISIÈME|QUATRIÈME|CINQUIÈME|SIXIÈME|SEPTIÈME|HUITIÈME|NEUVIÈME|DIXIÈME)\s*:?.*)", re.IGNORECASE)

text = """
9170 JOURNAL OFFICIEL DE LA REPUBLIQUE FRANÇAISE 5 Octobre 1958
Article 86.
La transformation du statut d’un Etat membre de la Communauté peut être demandée soit 
par la République, soit par une résolution de l’assemblée législative de l’Etat intéressé confirmée 
par un référendum local dont l’organisation et le contrôle sont assurés par les institutions de 
la Communauté. Les modalités de cette transformation sont déterminées par un accord approuvé 
par le Parlement de la République et l’assemblée législative intéressée.Dans les mêmes conditions, un Etat membre de la Communauté peut devenir indépendant. 
Il cesse de ce fait d’appartenir à la Communauté.  ,
Article 87.
Les accords particuliers conclus pour l’application du présent titre sont approuvés par le 
Parlement de la République et l’assemblée législative intéressée.TITRE XIIIr#ES ACCORDS D’ASSOCIATION 
Article 88.
La République ou la Communauté peuvent conclure des accords avec des Etats qui désirent 
s’associer à elle pour développer leurs civilisations.TITRE XIV
DE LA REVISIONArticle 89.
L’initiative de la révision de la Constitution appartient concurremment au Président de 
la République sur proposition du Premier Ministre et aux membres du Parlement.Le projet ou la proposition de révision doit être voté par les deux assemblées en termes 
identiques. La révision est définitive après avoir été approuvée par référendum.Toutefois, le projet de révision n’est pas présenté au référendum lorsque le Président de 
la République décide de le soumettre au Parlement convoqué en Congrès ; dans ce cas, le projet 
de révision n’est approuvé que s’il réunit la ,majorité des trois cinquièmes des suffrages 
exprimés. Le bureau du Congrès est celui de l’Assemblée Nationale.Aucune procédure de révision ne peut être engagée ou poursuivie lorsqu’il est porté atteinte 
à l’intégrité du territoire.La forme républicaine du Gouvernement ne peut faire l’objet d’une révision.
5 Octobre 1958 JOURNAL OFFICIEL DE LA REPUBLIQUE FRANÇAISE 9171
TITRE XV
DISPOSITIONS TRANSITOIRESArticle go,  '
La session ordinaire: du Parlement est suspendue. Le mandat des membres de l’Assemblée 
Nationale en fonction viendra à expiration le jour de la réunion de T Assemblée élue en vertu 
de la présente Constitution.Le Gouvernement, jusqu’à cette réunion, a seul autorité pour convoquer le Parlement.Le mandat des membres de l’Assemblée de l’Union Française viendra à expiration en même 
temps que le mandat des membres dë l’Assemblée Nationale actuellement en fonction.
Article gi.
Les institutions de la République prévues par la présente Constitution seront mises en 
place dans le délai de quatre mois à compter de sa promulgation.Ce délai est porté à six mois pour les institutions de la Communauté.Les pouvoirs du Président de la République en fonction ne viendront à expiration que 
lors de la proclamation des résultats de l’élection prévue par les articles 6 et 7 de la présente 
Constitution.Les Etats membres de la Communauté participeront à cette première élection dans les 
conditions découlant de leur statut à la date de la promulgation de la Constitution.Les autorités établies continueront d’exercer leurs fonctions dans ces Etats conformément 
aux lois et règlements applicables au moment de l’entrée en vigueur de la Constitution jusqu’à 
la mise en place des autorités prévues par leur nouveau régime.Jusqu’à sa constitution définitive, le Sénat est formé par les membres en fonction du Conseil 
de la République. Les lois organiques, qui régleront la constitution définitive du Sénat devront 
intervenir avant le 31 juillet 1959.Les attributions conférées au Conseil Constitutionnel par les articles 58 et 59 de la Consti­
tution seront exercées, jusqu’à la mise en place de ce Conseil, par une Commission composée du 
vice-président du Conseil d’Etat, président, du Premier Président de la Cour de Cassation et 
du Premier Président de la Cour des Comptes.Les peuples des Etats membres de la Communauté continuent à être représentés au Parle­
ment jusqu’à l’entrée en vigueur des mesures nécessaires à l’application du titre XII.
Article g2.
Les mesures législatives nécessaires à la mise en place des institutions et, jusqu’à cette 
mise en place, au fonctionnement des pouvoirs publics seront prises en Conseil des Ministres, 
après avis du Conseil d’Etat, par ordonnances ayant force de loi.

"""

parts = re.split(titre_pattern, text)
for p in parts[:20]:
    print("----")
    print(p)

----

9170 JOURNAL OFFICIEL DE LA REPUBLIQUE FRANÇAISE 5 Octobre 1958
Article 86.
La transformation du statut d’un Etat membre de la Communauté peut être demandée soit 
par la République, soit par une résolution de l’assemblée législative de l’Etat intéressé confirmée 
par un référendum local dont l’organisation et le contrôle sont assurés par les institutions de 
la Communauté. Les modalités de cette transformation sont déterminées par un accord approuvé 
par le Parlement de la République et l’assemblée législative intéressée.Dans les mêmes conditions, un Etat membre de la Communauté peut devenir indépendant. 
Il cesse de ce fait d’appartenir à la Communauté.  ,
Article 87.
Les accords particuliers conclus pour l’application du présent titre sont approuvés par le 
Parlement de la République et l’assemblée législative intéressée.TITRE XIIIr#ES ACCORDS D’ASSOCIATION 
Article 88.
La République ou la Communauté peuvent conclure des accords avec des Etats qui désirent 
s’associer à elle po